# Çok Değişkenli Tahmin için Ayrı SARIMA Modelleri

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import time
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import statsmodels.api as sm

## Adım 1: Veri Yükleme ve Hazırlama

Veriyi 30 dakikalık ortalamalara yeniden örnekleyeceğiz, 3 sütunu da koruyacağız.

In [2]:
# İşlenmiş veriyi oku
df = pd.read_csv('genel_sunucu_yuku_tum_metrikler.csv', index_col='timestamp')
df.index = pd.to_datetime(df.index, unit='s')

# Veriyi 30 dakikalık aralıklara yeniden örnekle
df_resampled = df.resample('30min').mean().dropna()

# Tahmin edilecek sütunlar
target_columns = ['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']
ts_data = df_resampled[target_columns]

# Veriyi eğitim ve test setlerine ayır (Son 1 gün test verisi)
new_test_size = 48
train_data = ts_data[:-new_test_size]
test_data = ts_data[-new_test_size:]

print(f"Eğitim verisi boyutu: {len(train_data)}")
print(f"Test verisi boyutu: {len(test_data)}")

Eğitim verisi boyutu: 1392
Test verisi boyutu: 48


## Adım 2: Her Metrik İçin Ayrı Model Kurma ve Eğitme

Şimdi, `target_columns` listesindeki her bir metrik için bir döngü oluşturup ayrı ayrı SARIMA modelleri eğiteceğiz.

In [3]:
# Model parametreleri (tüm modeller için aynı kullanılacak)
p, d, q = 1, 1, 1
P, D, Q, s = 1, 1, 1, 48 # 30 dakikalık veri için s=48

all_results = {}
start_time = time.time()

for column in target_columns:
    print(f"\n--- '{column}' için SARIMA modeli eğitiliyor... ---")
    
    # Modeli sadece ilgili sütun üzerinde eğit
    model = sm.tsa.SARIMAX(
        train_data[column], # Sadece o anki sütunu seç
        order=(p, d, q),
        seasonal_order=(P, D, Q, s),
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    
    result = model.fit()
    
    # Tahmin yap
    start_index = len(train_data)
    end_index = len(train_data) + len(test_data) - 1
    predictions = result.get_prediction(start=start_index, end=end_index)
    
    # Sonuçları sakla
    all_results[column] = {
        'predictions': predictions.predicted_mean,
        'model_summary': result.summary()
    }
    print(f"'{column}' için eğitim tamamlandı.")

end_time = time.time()
print(f"\nTüm modellerin eğitimi tamamlandı. Toplam süre: {end_time - start_time:.2f} saniye")


--- 'avg_min_cpu' için SARIMA modeli eğitiliyor... ---
'avg_min_cpu' için eğitim tamamlandı.

--- 'avg_max_cpu' için SARIMA modeli eğitiliyor... ---
'avg_max_cpu' için eğitim tamamlandı.

--- 'avg_avg_cpu' için SARIMA modeli eğitiliyor... ---
'avg_avg_cpu' için eğitim tamamlandı.

Tüm modellerin eğitimi tamamlandı. Toplam süre: 106.69 saniye


## Adım 3: Her Modelin Performansını Değerlendirme

In [4]:
# MAPE fonksiyonu
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100

print("--- SARIMA Modelleri İçin Performans Metrikleri ---")

for column in target_columns:
    y_true = test_data[column].values
    y_pred = all_results[column]['predictions'].values
    
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    
    print(f"\n--- Metrik: '{column}' ---")
    print(f"  Ortalama Mutlak Hata (MAE): {mae:.2f}")
    print(f"  Kök Ortalama Kare Hata (RMSE): {rmse:.2f}")
    print(f"  R-Kare (R²): {r2:.2f}")
    print(f"  Ortalama Mutlak Yüzde Hata (MAPE): {mape:.2f}%")

--- SARIMA Modelleri İçin Performans Metrikleri ---

--- Metrik: 'avg_min_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.13
  Kök Ortalama Kare Hata (RMSE): 0.15
  R-Kare (R²): -0.40
  Ortalama Mutlak Yüzde Hata (MAPE): 2.26%

--- Metrik: 'avg_max_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.47
  Kök Ortalama Kare Hata (RMSE): 0.55
  R-Kare (R²): 0.10
  Ortalama Mutlak Yüzde Hata (MAPE): 2.49%

--- Metrik: 'avg_avg_cpu' ---
  Ortalama Mutlak Hata (MAE): 0.27
  Kök Ortalama Kare Hata (RMSE): 0.31
  R-Kare (R²): -0.22
  Ortalama Mutlak Yüzde Hata (MAPE): 2.87%


## Adım 4: Sonuçları Görselleştirme

In [5]:
print("Tüm metrikler için sonuçlar görselleştiriliyor...")

fig = go.Figure()

# Her bir metrik için gerçek ve tahmin değerlerini çiz
for column in target_columns:
    # Gerçek Değerler (Test)
    fig.add_trace(go.Scatter(
        x=test_data.index,
        y=test_data[column],
        mode='lines',
        name=f'Gerçek {column}'
    ))
    
    # Model Tahminleri
    fig.add_trace(go.Scatter(
        x=all_results[column]['predictions'].index,
        y=all_results[column]['predictions'].values,
        mode='lines',
        name=f'Tahmin {column}',
        line=dict(dash='dash') # Tahmin çizgilerini kesikli yap
    ))

fig.update_layout(
    title='CPU Metrikleri: Gerçek Değerler vs. SARIMA Tahminleri',
    xaxis_title='Zaman',
    yaxis_title='Ortalama CPU Yükü (%)'
)

fig.show()

Tüm metrikler için sonuçlar görselleştiriliyor...
